In [1]:
import cv2 
import math
import numpy as np
import imutils
from numba import jit, cuda
from timeit import default_timer as timer

def find(full,face):
    if(len(face.shape)<3):
        height, width = face.shape
    elif(len(face.shape)==3):
        height, width,channels = face.shape
    methods = ['cv2.TM_CCOEFF']
    for m in methods:
        full_copy = full.copy()
        method = eval(m)
        res = cv2.matchTemplate(full_copy,face,method)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
            top_left = min_loc    
        else:
            top_left = max_loc
        bottom_right = (top_left[0] + width, top_left[1] + height)
        cv2.rectangle(full_copy,top_left, bottom_right, (0,0,255), 2)
        return full_copy[top_left[1]:top_left[1]+height,top_left[0]:top_left[0]+width,:]
    
@jit(target_backend='cuda')    
def detect_contour(img):
    mx_up=img.shape[0]
    mx_lt=img.shape[1]
    mx_rt=0
    mx_dn=0
    for j in range (img.shape[0]):
        g=0
        f=0
        for k in range (img.shape[1]):
            if(img[j,k]!=0):
                img[j,k]=255
                f+=1
        g=f
        f=0
        for k in range (img.shape[1]):
            if(img[j,k]!=0):
                img[j,k]=255
                f+=1
                if (f==1 and k<mx_lt):
                    mx_lt=k
                if (f==g and k>mx_rt):
                    mx_rt=k
    
    for j in range (img.shape[1]):
        g=0
        f=0
        for k in range (img.shape[0]):
            if(img[k,j]!=0):
                img[k,j]=255
                f+=1
        g=f
        f=0
        for k in range (img.shape[0]):
            if(img[k,j]!=0):
                img[k,j]=255
                f+=1
                if (f==1 and k<mx_up):
                    mx_up=k
                if (f==g and k>mx_dn):
                    mx_dn=k
    mx_lt-=1#9
    mx_dn+=2#10
    mx_rt+=2#10
    mx_up-=1#9
    
    #mx_up=0
    #mx_lt=0
    #mx_rt=img.shape[1]
    #mx_dn=img.shape[0]
    return img[mx_up:mx_dn,mx_lt:mx_rt]

def rotate(image,a,s=1.1):
    if(len(image.shape)<3):
        img1=image
        x,y=img1.shape
        X,Y=int(2.5*x),int(2.5*y)
        image = np.full((X,Y), 0, dtype=np.uint8)
        x0=int((X/2)-(img1.shape[0]/2))
        x1=int((X/2)+(img1.shape[0]/2))
        y0=int((Y/2)-(img1.shape[1]/2))
        y1=int((Y/2)+(img1.shape[1]/2))
        image[x0:x1,y0:y1]=img1

        height, width = image.shape[:2]
        center = (width/2, height/2)
        rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=a, scale=s)
        return cv2.warpAffine(src=image, M=rotate_matrix, dsize=(width, height))
    elif(len(image.shape)==3):
        img1=image
        x,y,z=img1.shape
        X,Y=int(2*x),int(2*y)
        image = np.full((X,Y,z), 0, dtype=np.uint8)
        x0=int((X/2)-(img1.shape[0]/2))
        x1=int((X/2)+(img1.shape[0]/2))
        y0=int((Y/2)-(img1.shape[1]/2))
        y1=int((Y/2)+(img1.shape[1]/2))
        image[x0:x1,y0:y1,:]=img1

        height, width = image.shape[:2]
        center = (width/2, height/2)
        rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=a, scale=s)
        return cv2.warpAffine(src=image, M=rotate_matrix, dsize=(width, height))
    
def find_angle(img1):
    angle=0
    mv=999999999
    b=9999999999
    x,y=img1.shape
    X,Y=int(1.5*x),int(1.5*y)
    img2 = np.full((X,Y), 0, dtype=np.uint8)
    x0=int((X/2)-(img1.shape[0]/2))
    x1=int((X/2)+(img1.shape[0]/2))
    y0=int((Y/2)-(img1.shape[1]/2))
    y1=int((Y/2)+(img1.shape[1]/2))
    img2[x0:x1,y0:y1]=img1
    for i in range(360):
        start = timer()
        img3=rotate(img2,i)
        img4=detect_contour(img3)
        #print(str(i), timer()-start)                   #timer
        h,w = img4.shape
        #print(str(i),'-',str(h*w),'-',str(w),'-',str(h))
        if(h*w<=mv):
            if(w<b):
                b=w
                mv=h*w
                angle=i
    return angle

def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    if(len(img1.shape)<3):
        if ((img1.shape[0])<(img2.shape[0])):
            X=img2.shape[0]
            Y=img1.shape[1]
            img3 = np.full((X,Y), 0, dtype=np.uint8)
            x0=int((X/2)-(img1.shape[0]/2))
            x1=int((X/2)+(img1.shape[0]/2))
            y0=int((Y/2)-(img1.shape[1]/2))
            y1=int((Y/2)+(img1.shape[1]/2))
            img3[x0:x1,y0:y1]=img1
            img1=img3
            
        if ((img1.shape[0])>(img2.shape[0])):
            X=img1.shape[0]
            Y=img2.shape[1]
            img3 = np.full((X,Y), 0, dtype=np.uint8)
            x0=int((X/2)-(img2.shape[0]/2))
            x1=int((X/2)+(img2.shape[0]/2))
            y0=int((Y/2)-(img2.shape[1]/2))
            y1=int((Y/2)+(img2.shape[1]/2))
            img3[x0:x1,y0:y1]=img2
            img2=img3
            
        if ((img1.shape[1])<(img2.shape[1])):
            X=img1.shape[0]
            Y=img2.shape[1]
            img3 = np.full((X,Y), 0, dtype=np.uint8)
            x0=int((X/2)-(img1.shape[0]/2))
            x1=int((X/2)+(img1.shape[0]/2))
            y0=int((Y/2)-(img1.shape[1]/2))
            y1=int((Y/2)+(img1.shape[1]/2))
            img3[x0:x1,y0:y1]=img1
            img1=img3
            
        if ((img1.shape[1])>(img2.shape[1])):
            X=img2.shape[0]
            Y=img1.shape[1]
            img3 = np.full((X,Y), 0, dtype=np.uint8)
            x0=int((X/2)-(img2.shape[0]/2))
            x1=int((X/2)+(img2.shape[0]/2))
            y0=int((Y/2)-(img2.shape[1]/2))
            y1=int((Y/2)+(img2.shape[1]/2))
            img3[x0:x1,y0:y1]=img2
            img2=img3
    
    elif(len(img1.shape)==3):
        if ((img1.shape[0])<(img2.shape[0])):
            X=img2.shape[0]
            Y=img1.shape[1]
            img3 = np.full((X,Y,3), 0, dtype=np.uint8)
            x0=int((X/2)-(img1.shape[0]/2))
            x1=int((X/2)+(img1.shape[0]/2))
            y0=int((Y/2)-(img1.shape[1]/2))
            y1=int((Y/2)+(img1.shape[1]/2))
            img3[x0:x1,y0:y1,:]=img1
            img1=img3
            
        if ((img1.shape[0])>(img2.shape[0])):
            X=img1.shape[0]
            Y=img2.shape[1]
            img3 = np.full((X,Y,3), 0, dtype=np.uint8)
            x0=int((X/2)-(img2.shape[0]/2))
            x1=int((X/2)+(img2.shape[0]/2))
            y0=int((Y/2)-(img2.shape[1]/2))
            y1=int((Y/2)+(img2.shape[1]/2))
            img3[x0:x1,y0:y1,:]=img2
            img2=img3
            
        if ((img1.shape[1])<(img2.shape[1])):
            X=img1.shape[0]
            Y=img2.shape[1]
            img3 = np.full((X,Y,3), 0, dtype=np.uint8)
            x0=int((X/2)-(img1.shape[0]/2))
            x1=int((X/2)+(img1.shape[0]/2))
            y0=int((Y/2)-(img1.shape[1]/2))
            y1=int((Y/2)+(img1.shape[1]/2))
            img3[x0:x1,y0:y1,:]=img1
            img1=img3
            
        if ((img1.shape[1])>(img2.shape[1])):
            X=img2.shape[0]
            Y=img1.shape[1]
            img3 = np.full((X,Y,3), 0, dtype=np.uint8)
            x0=int((X/2)-(img2.shape[0]/2))
            x1=int((X/2)+(img2.shape[0]/2))
            y0=int((Y/2)-(img2.shape[1]/2))
            y1=int((Y/2)+(img2.shape[1]/2))
            img3[x0:x1,y0:y1,:]=img2
            img2=img3
            
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))

def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calculate_ssim(img1, img2):
    if(len(img1.shape)<3):
        if ((img1.shape[0])<(img2.shape[0])):
            X=img2.shape[0]
            Y=img1.shape[1]
            img3 = np.full((X,Y), 0, dtype=np.uint8)
            x0=int((X/2)-(img1.shape[0]/2))
            x1=int((X/2)+(img1.shape[0]/2))
            y0=int((Y/2)-(img1.shape[1]/2))
            y1=int((Y/2)+(img1.shape[1]/2))
            img3[x0:x1,y0:y1]=img1
            img1=img3
            
        if ((img1.shape[0])>(img2.shape[0])):
            X=img1.shape[0]
            Y=img2.shape[1]
            img3 = np.full((X,Y), 0, dtype=np.uint8)
            x0=int((X/2)-(img2.shape[0]/2))
            x1=int((X/2)+(img2.shape[0]/2))
            y0=int((Y/2)-(img2.shape[1]/2))
            y1=int((Y/2)+(img2.shape[1]/2))
            img3[x0:x1,y0:y1]=img2
            img2=img3
            
        if ((img1.shape[1])<(img2.shape[1])):
            X=img1.shape[0]
            Y=img2.shape[1]
            img3 = np.full((X,Y), 0, dtype=np.uint8)
            x0=int((X/2)-(img1.shape[0]/2))
            x1=int((X/2)+(img1.shape[0]/2))
            y0=int((Y/2)-(img1.shape[1]/2))
            y1=int((Y/2)+(img1.shape[1]/2))
            img3[x0:x1,y0:y1]=img1
            img1=img3
            
        if ((img1.shape[1])>(img2.shape[1])):
            X=img2.shape[0]
            Y=img1.shape[1]
            img3 = np.full((X,Y), 0, dtype=np.uint8)
            x0=int((X/2)-(img2.shape[0]/2))
            x1=int((X/2)+(img2.shape[0]/2))
            y0=int((Y/2)-(img2.shape[1]/2))
            y1=int((Y/2)+(img2.shape[1]/2))
            img3[x0:x1,y0:y1]=img2
            img2=img3
    
    elif(len(img1.shape)==3):
        if ((img1.shape[0])<(img2.shape[0])):
            X=img2.shape[0]
            Y=img1.shape[1]
            img3 = np.full((X,Y,3), 0, dtype=np.uint8)
            x0=int((X/2)-(img1.shape[0]/2))
            x1=int((X/2)+(img1.shape[0]/2))
            y0=int((Y/2)-(img1.shape[1]/2))
            y1=int((Y/2)+(img1.shape[1]/2))
            img3[x0:x1,y0:y1,:]=img1
            img1=img3
            
        if ((img1.shape[0])>(img2.shape[0])):
            X=img1.shape[0]
            Y=img2.shape[1]
            img3 = np.full((X,Y,3), 0, dtype=np.uint8)
            x0=int((X/2)-(img2.shape[0]/2))
            x1=int((X/2)+(img2.shape[0]/2))
            y0=int((Y/2)-(img2.shape[1]/2))
            y1=int((Y/2)+(img2.shape[1]/2))
            img3[x0:x1,y0:y1,:]=img2
            img2=img3
            
        if ((img1.shape[1])<(img2.shape[1])):
            X=img1.shape[0]
            Y=img2.shape[1]
            img3 = np.full((X,Y,3), 0, dtype=np.uint8)
            x0=int((X/2)-(img1.shape[0]/2))
            x1=int((X/2)+(img1.shape[0]/2))
            y0=int((Y/2)-(img1.shape[1]/2))
            y1=int((Y/2)+(img1.shape[1]/2))
            img3[x0:x1,y0:y1,:]=img1
            img1=img3
            
        if ((img1.shape[1])>(img2.shape[1])):
            X=img2.shape[0]
            Y=img1.shape[1]
            img3 = np.full((X,Y,3), 0, dtype=np.uint8)
            x0=int((X/2)-(img2.shape[0]/2))
            x1=int((X/2)+(img2.shape[0]/2))
            y0=int((Y/2)-(img2.shape[1]/2))
            y1=int((Y/2)+(img2.shape[1]/2))
            img3[x0:x1,y0:y1,:]=img2
            img2=img3
            
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')
        

def find_cntr(name):
    """
    cap = cv2.VideoCapture(1)

    ret, img_o = cap.read()
    cv2.imwrite('snapshot.png', img_o)
    img = cv2.imread('snapshot.png')
    """
    path='dataset/Training/Defect-free/DF ('+str(name)+').jpg'
    path_1='dataset_new/IDEAL1.jpg'
    path_2='dataset_new/IDEAL2.jpg'
    img_o=cv2.imread(path)
    img = cv2.imread(path)
    ideal1=cv2.imread('dataset_new/IDEAL1.jpg')
    ideal2=cv2.imread('dataset_new/IDEAL2.jpg')
    
    med_val = np.median(img)
    blurred_img = cv2.blur(img,ksize=(3,3))
    lower = int(max(0, 0.1* med_val))
    upper = int(min(255,1.9 * med_val))
    edges = cv2.Canny(image=blurred_img, threshold1=lower , threshold2=upper)
    cnts = cv2.findContours(edges, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    edges=img
    i=0
    n=0
    mxa=0
    x,y,z=img.shape
    external_contours = np.zeros((x,y,z))
    cl=len(cnts)
    #print(cl)
    for c in cnts:
        external_contours_centres = np.zeros((x,y))
        M = cv2.moments(c)
        if M["m00"] != 0:
            i+=1
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            cv2.drawContours(external_contours, [c], -1, (255, 255, 255), 1)
            cv2.drawContours(external_contours_centres, [c], -1, (255,255,255), 1)
            external_contours_centres=detect_contour(external_contours_centres)
            if(external_contours_centres.shape[0]*external_contours_centres.shape[1]>=mxa):
                mxa=external_contours_centres.shape[0]*external_contours_centres.shape[1]
                n=i
            
            
    #print(n)
    i=0
    external_contours_centres = np.zeros((x,y))
    for c in cnts:
        M = cv2.moments(c)
        if M["m00"] != 0:
            i+=1
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            if(i==n):
                cv2.drawContours(external_contours_centres, [c], -1, (255,255,255), 1)
            
    
    cv2.namedWindow("original", cv2.WINDOW_NORMAL)        
    cv2.resizeWindow("original", 630, 455)
    cv2.namedWindow("processed1", cv2.WINDOW_NORMAL)        
    cv2.resizeWindow("processed1", 630, 455)
    cv2.namedWindow("processed2", cv2.WINDOW_NORMAL)        
    cv2.resizeWindow("processed2", 630, 455)
    cv2.namedWindow("processed3", cv2.WINDOW_NORMAL)        
    cv2.resizeWindow("processed3", 630, 455)

    
    i=find_angle(detect_contour(external_contours_centres))
    #print(i)
    
    cv2.imwrite('original.jpg',rotate(external_contours,i))
    cv2.imwrite('match.png',detect_contour(rotate(detect_contour(external_contours_centres),i)))
    full = cv2.imread('original.jpg')
    face = cv2.imread('match.png')
    output=find(full,face)
    cv2.imwrite('output.jpg',output)
    output=cv2.imread('output.jpg',0)
    
    # sim1=calculate_ssim(output,ideal1)
    # sim2=calculate_ssim(output,ideal2)
#     if(sim1>=sim2):
#         sim=sim1
#         ideal=ideal1
#     else:
#         sim=sim2
#         ideal=ideal2
        
#     if(sim<0.8920000000000000):
#         print("Defective",sim)
#     else:
#         print("Defect-free",sim)
    
    
    cv2.imshow('original',img_o)
    cv2.imshow('processed1',rotate(img_o,i))
    cv2.imshow('processed2', ideal)
    cv2.imshow('processed3', output)
    
    cv2.waitKey(0)
    #cap.release()
    cv2.destroyAllWindows()

start = timer()
find_cntr(6)
print(timer()-start)


11.818671600000002


In [ ]:
#"""
start = timer()
for i in range(0,200):
    print(i)
    find_cntr(i)

print(timer()-start)
#"""

In [ ]:
import cv2 
import numpy as np
import imutils
from numba import jit, cuda
from timeit import default_timer as timer

def find(full,face):
    if(len(face.shape)<3):
        height, width = face.shape
    elif(len(face.shape)==3):
        height, width,channels = face.shape
    methods = ['cv2.TM_CCOEFF']
    for m in methods:
        full_copy = full.copy()
        method = eval(m)
        res = cv2.matchTemplate(full_copy,face,method)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
            top_left = min_loc    
        else:
            top_left = max_loc
        bottom_right = (top_left[0] + width, top_left[1] + height)
        cv2.rectangle(full_copy,top_left, bottom_right, (0,0,255), 1)
        return full_copy
    
@jit(target_backend='cuda')    
def detect_contour(img):
    mx_up=img.shape[0]
    mx_lt=img.shape[1]
    mx_rt=0
    mx_dn=0
    for j in range (img.shape[0]):
        g=0
        f=0
        for k in range (img.shape[1]):
            if(img[j,k]!=0):
                img[j,k]=255
                f+=1
        g=f
        f=0
        for k in range (img.shape[1]):
            if(img[j,k]!=0):
                img[j,k]=255
                f+=1
                if (f==1 and k<mx_lt):
                    mx_lt=k
                if (f==g and k>mx_rt):
                    mx_rt=k
    
    for j in range (img.shape[1]):
        g=0
        f=0
        for k in range (img.shape[0]):
            if(img[k,j]!=0):
                img[k,j]=255
                f+=1
        g=f
        f=0
        for k in range (img.shape[0]):
            if(img[k,j]!=0):
                img[k,j]=255
                f+=1
                if (f==1 and k<mx_up):
                    mx_up=k
                if (f==g and k>mx_dn):
                    mx_dn=k
    mx_lt-=1#9
    mx_dn+=2#10
    mx_rt+=2#10
    mx_up-=1#9
    
    #mx_up=0
    #mx_lt=0
    #mx_rt=img.shape[1]
    #mx_dn=img.shape[0]
    return img[mx_up:mx_dn,mx_lt:mx_rt]

def rotate(image,a,s=1.1):
    if(len(image.shape)<3):
        img1=image
        x,y=img1.shape
        X,Y=int(2.5*x),int(2.5*y)
        image = np.full((X,Y), 0, dtype=np.uint8)
        x0=int((X/2)-(img1.shape[0]/2))
        x1=int((X/2)+(img1.shape[0]/2))
        y0=int((Y/2)-(img1.shape[1]/2))
        y1=int((Y/2)+(img1.shape[1]/2))
        image[x0:x1,y0:y1]=img1

        height, width = image.shape[:2]
        center = (width/2, height/2)
        rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=a, scale=s)
        return cv2.warpAffine(src=image, M=rotate_matrix, dsize=(width, height))
    elif(len(image.shape)==3):
        img1=image
        x,y,z=img1.shape
        X,Y=int(2*x),int(2*y)
        image = np.full((X,Y,z), 0, dtype=np.uint8)
        x0=int((X/2)-(img1.shape[0]/2))
        x1=int((X/2)+(img1.shape[0]/2))
        y0=int((Y/2)-(img1.shape[1]/2))
        y1=int((Y/2)+(img1.shape[1]/2))
        image[x0:x1,y0:y1,:]=img1

        height, width = image.shape[:2]
        center = (width/2, height/2)
        rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=a, scale=s)
        return cv2.warpAffine(src=image, M=rotate_matrix, dsize=(width, height))
    
def find_angle(img1):
    angle=0
    mv=999999999
    b=9999999999
    x,y=img1.shape
    X,Y=int(1.5*x),int(1.5*y)
    img2 = np.full((X,Y), 0, dtype=np.uint8)
    x0=int((X/2)-(img1.shape[0]/2))
    x1=int((X/2)+(img1.shape[0]/2))
    y0=int((Y/2)-(img1.shape[1]/2))
    y1=int((Y/2)+(img1.shape[1]/2))
    img2[x0:x1,y0:y1]=img1
    for i in range(360):
        start = timer()
        img3=rotate(img2,i)
        img4=detect_contour(img3)
        #print(str(i), timer()-start)                   #timer
        h,w = img4.shape
        #print(str(i),'-',str(h*w),'-',str(w),'-',str(h))
        if(h*w<mv):
            if(w<b):
                b=w
                mv=h*w
                angle=i
    return angle

def find_cntr(name,ni):
    """
    cap = cv2.VideoCapture(1)

    ret, img_o = cap.read()
    cv2.imwrite('snapshot.png', img_o)
    img = cv2.imread('snapshot.png')
    """
    path='dataset/Training/Defect-free/DF ('+str(name)+').jpg'
    img_o=cv2.imread(path)
    img = cv2.imread(path)
    
    med_val = np.median(img)
    blurred_img = cv2.blur(img,ksize=(3,3))
    lower = int(max(0, 0.1* med_val))
    upper = int(min(255,1.9 * med_val))
    edges = cv2.Canny(image=blurred_img, threshold1=lower , threshold2=upper)
    cnts = cv2.findContours(edges, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    i=0
    n=0
    mxa=0
    x,y,z=img.shape
    external_contours = np.zeros((x,y,z))
    cl=len(cnts)
    print(cl)
    for c in cnts:
        external_contours_centres = np.zeros((x,y))
        M = cv2.moments(c)
        if M["m00"] != 0:
            i+=1
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            cv2.drawContours(external_contours, [c], -1, (255, 255, 255), 1)
            cv2.drawContours(external_contours_centres, [c], -1, (255,255,255), 1)
            external_contours_centres=detect_contour(external_contours_centres)
        if(external_contours_centres.shape[0]*external_contours_centres.shape[1]>mxa):
            mxa=external_contours_centres.shape[0]*external_contours_centres.shape[1]
            n=i
            
    print(n)
    i=0
    external_contours_centres = np.zeros((x,y))
    for c in cnts:
        M = cv2.moments(c)
        if M["m00"] != 0:
            i+=1
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            if(i==ni):
                cv2.drawContours(external_contours_centres, [c], -1, (255,255,255), 1)
            
    
    cv2.namedWindow("original", cv2.WINDOW_NORMAL)        
    cv2.resizeWindow("original", 630, 455)
    cv2.namedWindow("processed1", cv2.WINDOW_NORMAL)        
    cv2.resizeWindow("processed1", 630, 455)
    cv2.namedWindow("processed2", cv2.WINDOW_NORMAL)        
    cv2.resizeWindow("processed2", 630, 455)
    cv2.namedWindow("processed3", cv2.WINDOW_NORMAL)        
    cv2.resizeWindow("processed3", 630, 455)
    
    
    i=find_angle(detect_contour(external_contours_centres))
    print(i)
    
    
    cv2.imwrite('original.jpg',rotate(external_contours,i))
    cv2.imwrite('match.png',detect_contour(rotate(detect_contour(external_contours_centres),i)))
    full = cv2.imread('original.jpg')
    face = cv2.imread('match.png')
    
    cv2.imshow('original',img_o)
    cv2.imshow('processed1',rotate(img_o,i))
    cv2.imshow('processed2', rotate(external_contours_centres,i))
    cv2.imshow('processed3', rotate(external_contours,i))
    
    cv2.waitKey(0)
    #cap.release()
    cv2.destroyAllWindows()
    
start = timer()
find_cntr(100,9)
print(timer()-start)

650
